In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
import os
import imageio
import pandas

In [ ]:
relativisticEnergy = 105000 #keV

restMass = 511 #keV

initialGamma = relativisticEnergy/restMass

initialBeta = (1-(1/initialGamma)**2)**(1/2)

theta = np.pi/4

betaPerpendicular = initialBeta * np.sin(theta)

betaParallel = initialBeta * np.cos(theta)

magField = 0.34626038781 #keV^2

omega = magField/(initialGamma * restMass) #keV

helixRadius = betaPerpendicular/omega #1/keV

def gammaList():
    gammaLst = [initialGamma]
    time = 0
    helixRadiusUpdate = helixRadius
    gammaUpdate = initialGamma
    energy = relativisticEnergy
    betaUpdate = initialBeta
    newBetaPerpendicular = betaPerpendicular
    newBetaParallel = betaParallel
    omegaUpdate = omega
    while(energy > 0):
        dt = 1.0*10**5 #1/keV
        omega_c = magField/restMass
        power = ((2*omega_c**2)/3)*(newBetaPerpendicular**2/(1-betaUpdate**2))
        emittedRadiation = ((2*omega_c**2)/3)*(newBetaPerpendicular**2/(1-betaUpdate**2)) * dt
        time += dt
        energy = energy - emittedRadiation
        gammaUpdate = energy/restMass
        gammaLst.append(gammaUpdate)
        betaUpdate = (1-(1/(gammaUpdate))**2)**(1/2)
        newBetaPerpendicular = betaUpdate * np.sin(theta)
        newBetaParallel = betaUpdate * np.cos(theta)
        omegaUpdate = magField/(gammaUpdate * restMass)
        helixRadiusUpdate = newBetaPerpendicular/omegaUpdate
        if emittedRadiation < 1:
            break
        
    return gammaLst 

#print(gammaList())
#print(f'this is the length of gamma list: {len(gammaList())}')

def angDistPerGamma():
    gammas = gammaList()
    filenames = []
    for g in gammas[0:20]:
        thetas = np.arange(0, 2*np.pi, 0.01)
        results = []
        for ang in thetas:
            beta = (1-(1/(g))**2)**(1/2)
            betaPerp = beta * np.sin(ang)
            betaParallel = beta * np.cos(ang)
            g_parallel = 1 - (betaParallel * np.cos(ang))
            f_num = 4*(g_parallel**2)*( ((1 + betaParallel**2)*(1+(np.cos(ang))**2)) 
                                    - 4*betaParallel*np.cos(ang)) - (1 - betaParallel**2 + 3*(betaPerp**2))*(betaPerp**2)*(np.sin(ang))**4
            f_denom = 4*(g_parallel**2 - (betaPerp)**2*(np.sin(ang))**2)**(7/2)
            f = f_num/f_denom
            g_eqn = (3/4) * (1-beta**2)**2*f
            results.append(g_eqn)
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        ax.plot(thetas, results)
        m = np.max(results) * 1.5
        ax.set_rmax(m)
        ax.set_rticks([m/4, m/2, m*0.75, m])  # Less radial ticks
        ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
        ax.grid(True)
        filenames.append(plt.savefig(f'{g}.png'))
        plt.close()
    return filenames
filenames = angDistPerGamma()
with imageio.get_writer('mygif.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image) 
for filename in set(filenanmes):
    os.remove(filename)



    

In [7]:
len(gammaList())

3957